In [1]:
from  ibm_watson_machine_learning import APIClient 

In [2]:
import json 
import numpy as np 

In [9]:
import os 

In [3]:
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com", 
    "apikey": "06OUEcO3iWSP-hDf_yAKIR3lE6MS6RTSmLFlmbtjNDd3"  # "50tndXfHZWBvTOYzo-IG1MOK6LQAsSNObk0XgbdkBvSW" 
}

In [4]:
wml_client = APIClient(wml_credentials) 

In [5]:
wml_client.spaces.list() 

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------  ------------------------
ID                                    NAME          CREATED
6c81a779-0c13-4fbf-8a92-7977342d693d  lipNet_space  2023-08-30T12:49:15.298Z
------------------------------------  ------------  ------------------------


ID          NAME  \
0  6c81a779-0c13-4fbf-8a92-7977342d693d  lipNet_space   

                    CREATED  
0  2023-08-30T12:49:15.298Z

In [6]:
SPACE_ID = "6c81a779-0c13-4fbf-8a92-7977342d693d"  

In [7]:
wml_client.set.default_space(SPACE_ID) 

'SUCCESS'

In [12]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten 
import os 


vocab = "abcdefghijklmnopqrstuvwxyz'?!123456789 " 
vocab = [x for x in vocab] 

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token = "") 
num_to_char = tf.keras.layers.StringLookup(vocabulary = char_to_num.get_vocabulary(), oov_token = "", invert = True)  

# MODEL 

# vocab = "abcdefghijklmnopqrstuvwxyz'?!123456789 " 
# vocab = [x for x in vocab] 

# char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token = "") 
# num_to_char = tf.keras.layers.StringLookup(vocabulary = char_to_num.get_vocabulary(), oov_token = "", invert = True)  


def LipNet(): 
    model = Sequential()
    model.add(Conv3D(128, 3, input_shape=(75,46,140,1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))
    
    model.add(Conv3D(256, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))
    
    model.add(Conv3D(75, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))
    
    model.add(TimeDistributed(Flatten()))
    
    model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(.5))
    
    model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(.5))
    
    model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax')) 
    
    #model.load_weights("../models/best_model/checkpoint") 
       
    return model 

In [13]:
model = LipNet() 

In [16]:
model.load_weights("./models/best_model/checkpoint") 

In [10]:
os.listdir()  

['.ipynb_checkpoints',
 'Dataset',
 'LipNet.h5',
 'LipNet.tgz',
 'MLUtils',
 'models',
 'Test',
 'watson_deployment.ipynb',
 'WebApp',
 '__pycache__']

In [17]:
model 

In [11]:
MODEL_NAME = "LipNet Tensorflow" 
DEPLOYMENT_NAME = "LipNet Deployment"  

In [12]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")    

In [13]:
software_spec_uid

'b56101f1-309d-549b-a849-eaa63f77b2fb'

In [14]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: "tensorflow_2.9",   
    #wml_client.repository.ModelMetaNames.FRAMEWORK_NAME:"tensorflow", 
    #wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "2.10",  
    #wml_client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES:[{'name':'keras', 'version': '2.4'}],  
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [15]:
best_model = "./LipNet.tgz" 

In [16]:
wml_client.deployments.get_details() 

{'resources': []}

In [17]:
model_details = wml_client.repository.store_model(
    model = best_model,   
    meta_props = model_props 
) 

In [18]:
wml_client.repository.list_models()  

------------------------------------  -----------------  ------------------------  --------------  ----------  ----------------
ID                                    NAME               CREATED                   TYPE            SPEC_STATE  SPEC_REPLACEMENT
07705a9a-7dc0-432f-8375-9c1f1b0ece9f  LipNet Tensorflow  2023-08-30T13:02:17.002Z  tensorflow_2.9  supported
------------------------------------  -----------------  ------------------------  --------------  ----------  ----------------


ID               NAME  \
0  07705a9a-7dc0-432f-8375-9c1f1b0ece9f  LipNet Tensorflow   

                    CREATED            TYPE SPEC_STATE SPEC_REPLACEMENT  
0  2023-08-30T13:02:17.002Z  tensorflow_2.9  supported

In [19]:
model_details 

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'b56101f1-309d-549b-a849-eaa63f77b2fb',
   'name': 'runtime-22.2-py3.10'},
  'type': 'tensorflow_2.9'},
 'metadata': {'created_at': '2023-08-30T13:02:17.573Z',
  'id': '07705a9a-7dc0-432f-8375-9c1f1b0ece9f',
  'modified_at': '2023-08-30T13:02:29.931Z',
  'name': 'LipNet Tensorflow',
  'owner': 'IBMid-6680017HMM',
  'resource_key': 'b1e7be61-d74f-43fe-b3db-5211a8c41244',
  'space_id': '6c81a779-0c13-4fbf-8a92-7977342d693d'},
 'system': {'warnings': []}}

In [20]:
model_uid = model_details["metadata"]["id"] 

In [21]:
model_uid 

'07705a9a-7dc0-432f-8375-9c1f1b0ece9f'

In [22]:
# deployment Props 
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {} 
}

#actual deployment 
deployment = wml_client.deployments.create(
    artifact_uid = model_uid, 
    meta_props = deployment_props 
) 



#######################################################################################

Synchronous deployment creation for uid: '07705a9a-7dc0-432f-8375-9c1f1b0ece9f' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5015465f-770f-459a-9a06-c071fd198ffd'
------------------------------------------------------------------------------------------------




In [23]:
deployment 

{'entity': {'asset': {'id': '07705a9a-7dc0-432f-8375-9c1f1b0ece9f'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'f3ebac7d-0a75-410c-8b48-a931428cc4c5',
   'name': 'XS',
   'num_nodes': 1},
  'name': 'LipNet Deployment',
  'online': {},
  'space_id': '6c81a779-0c13-4fbf-8a92-7977342d693d',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/5015465f-770f-459a-9a06-c071fd198ffd/predictions'},
   'serving_urls': ['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/5015465f-770f-459a-9a06-c071fd198ffd/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2023-08-30T13:03:39.279Z',
  'id': '5015465f-770f-459a-9a06-c071fd198ffd',
  'modified_at': '2023-08-30T13:03:39.279Z',
  'name': 'LipNet Deployment',
  'owner': 'IBMid-6680017HMM',
  'space_id': '6c81a779-0c13-4fbf-8a92-7977342d693d'},
 'system': {'warnings': [{'id': 'Deprecated',
    'message': 'online_url is deprecated and will be removed in a future rele

# Converting Model into .h5 Format and into tgz format 

In [27]:
os.listdir() 

['.ipynb_checkpoints',
 'Dataset',
 'deployment.ipynb',
 'MLUtils',
 'models',
 'Test',
 'WebApp',
 '__pycache__']

In [28]:
model.save("LipNet.h5") 

In [29]:
os.listdir() 

['.ipynb_checkpoints',
 'Dataset',
 'deployment.ipynb',
 'LipNet.h5',
 'MLUtils',
 'models',
 'Test',
 'WebApp',
 '__pycache__']

In [31]:
!tar -zcvf LipNet.tgz LipNet.h5 

a LipNet.h5


In [32]:
os.listdir() 

['.ipynb_checkpoints',
 'Dataset',
 'deployment.ipynb',
 'LipNet.h5',
 'LipNet.tgz',
 'MLUtils',
 'models',
 'Test',
 'WebApp',
 '__pycache__']

In [34]:
tf.keras.__version__ 

'2.10.0'

In [35]:
tf.__version__ 

'2.10.0'